# Практика с лекции

In [48]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [49]:
!curl https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv -O titanic.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 60302  100 60302    0     0   117k      0 --:--:-- --:--:-- --:--:--  117k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:03 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:04 --:--:--     0
  0     0    0     0    0     0      0      0 --:-

In [50]:
df = spark.read.csv('titanic.csv', inferSchema=True, header=True)

In [51]:
df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [52]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [53]:
df_filtered = df.select('Survived', 'Pclass', 'Sex', 'Age', 'Fare', 'Embarked')

In [54]:
df_filtered.show(5)

+--------+------+------+----+-------+--------+
|Survived|Pclass|   Sex| Age|   Fare|Embarked|
+--------+------+------+----+-------+--------+
|       0|     3|  male|22.0|   7.25|       S|
|       1|     1|female|38.0|71.2833|       C|
|       1|     3|female|26.0|  7.925|       S|
|       1|     1|female|35.0|   53.1|       S|
|       0|     3|  male|35.0|   8.05|       S|
+--------+------+------+----+-------+--------+
only showing top 5 rows



In [55]:
df_filtered.groupBy('Embarked').count().show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       Q|   77|
|    null|    2|
|       C|  168|
|       S|  644|
+--------+-----+



In [56]:
df_filtered.describe().show()

+-------+-------------------+------------------+------+------------------+-----------------+--------+
|summary|           Survived|            Pclass|   Sex|               Age|             Fare|Embarked|
+-------+-------------------+------------------+------+------------------+-----------------+--------+
|  count|                891|               891|   891|               714|              891|     889|
|   mean| 0.3838383838383838| 2.308641975308642|  null| 29.69911764705882| 32.2042079685746|    null|
| stddev|0.48659245426485753|0.8360712409770491|  null|14.526497332334035|49.69342859718089|    null|
|    min|                  0|                 1|female|              0.42|              0.0|       C|
|    max|                  1|                 3|  male|              80.0|         512.3292|       S|
+-------+-------------------+------------------+------+------------------+-----------------+--------+



In [57]:
df_filtered = df_filtered.na.fill({'Age':30,'Embarked':'S'})

In [58]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

In [59]:
indexer = StringIndexer(inputCol='Sex', outputCol='SexInd')
indexerTrained = indexer.fit(df_filtered)
df_upd = indexerTrained.transform(df_filtered)

indexer = StringIndexer(inputCol='Embarked', outputCol='EmbarkedInd')
indexerTrained = indexer.fit(df_upd)
df_upd = indexerTrained.transform(df_upd)

In [60]:
onehot = OneHotEncoder(inputCol='EmbarkedInd', outputCol='EmbarkedOhe')
onehotTrained = onehot.fit(df_upd)
df_upd = onehotTrained.transform(df_upd)

In [61]:
df_upd.show(5)

+--------+------+------+----+-------+--------+------+-----------+-------------+
|Survived|Pclass|   Sex| Age|   Fare|Embarked|SexInd|EmbarkedInd|  EmbarkedOhe|
+--------+------+------+----+-------+--------+------+-----------+-------------+
|       0|     3|  male|22.0|   7.25|       S|   0.0|        0.0|(2,[0],[1.0])|
|       1|     1|female|38.0|71.2833|       C|   1.0|        1.0|(2,[1],[1.0])|
|       1|     3|female|26.0|  7.925|       S|   1.0|        0.0|(2,[0],[1.0])|
|       1|     1|female|35.0|   53.1|       S|   1.0|        0.0|(2,[0],[1.0])|
|       0|     3|  male|35.0|   8.05|       S|   0.0|        0.0|(2,[0],[1.0])|
+--------+------+------+----+-------+--------+------+-----------+-------------+
only showing top 5 rows



In [47]:
from pyspark.ml.feature import VectorAssembler

In [62]:
assembler = VectorAssembler(inputCols=['Pclass','Age','Fare','SexInd','EmbarkedOhe'], outputCol='Features')

In [63]:
df_upd = assembler.transform(df_upd)
df_upd.show(5)

+--------+------+------+----+-------+--------+------+-----------+-------------+--------------------+
|Survived|Pclass|   Sex| Age|   Fare|Embarked|SexInd|EmbarkedInd|  EmbarkedOhe|            Features|
+--------+------+------+----+-------+--------+------+-----------+-------------+--------------------+
|       0|     3|  male|22.0|   7.25|       S|   0.0|        0.0|(2,[0],[1.0])|[3.0,22.0,7.25,0....|
|       1|     1|female|38.0|71.2833|       C|   1.0|        1.0|(2,[1],[1.0])|[1.0,38.0,71.2833...|
|       1|     3|female|26.0|  7.925|       S|   1.0|        0.0|(2,[0],[1.0])|[3.0,26.0,7.925,1...|
|       1|     1|female|35.0|   53.1|       S|   1.0|        0.0|(2,[0],[1.0])|[1.0,35.0,53.1,1....|
|       0|     3|  male|35.0|   8.05|       S|   0.0|        0.0|(2,[0],[1.0])|[3.0,35.0,8.05,0....|
+--------+------+------+----+-------+--------+------+-----------+-------------+--------------------+
only showing top 5 rows

